# Data paths

In [171]:
 # Sarah paths
#path_sat="/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SARAH_DATA/DNI/ORD49156"
path_sat="/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SARAH_DATA/SIS/ORD49425"
# Era5 
path_era="/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/ERA5_DATA/"
# On site data:   
path_site='/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SCRIPTS/ONSITE_DATA'
prefix = "DATA_SK"  # The prefix you want to match


## Solar Kapitla names 
# names=( agioi10 Mavroneri Sitia  Assiros Palaiokastro  Larisa Mesa1 Arnissa Radi Lagos )
# names working: agioi10, Sitia, Palaiokastro, Lagos 
locs= [ "Sitia" ]

In [172]:
# This sctipt merges Sarah csv data and data from solar kapital platforms 
import os 
import pandas as pd 
import csv
import re

# Read the ERA5 Land data 

In [173]:
## Convert all files from tsv to csv and save them locally with _COMMA.csv 
#locs= [ "Lagos" , "mavroneri" ]
names = [ "Tmax_2T_", "Tmean_2T_", "Tmin_2T_" ]
for loc in locs: 
    for name in names: 
        fname1= name+loc+".csv"
        fname1b= name+loc+"_COMMA.csv"
        #print(fname1)
        path1=os.path.join(path_era,fname1 )
        path1_csv = os.path.join(path_era,fname1b )
        #print(path1)
        #print(path1_csv)
        # Convert to comma delimited csv 
        with open(path1, 'r') as tsvfile, open(path1_csv, 'w', newline='') as csvfile:
                #print(tsvfile, name, loc)
                #print(csvfile, name, loc)
                tsvreader = csv.reader(tsvfile, delimiter='\t')
                csvwriter = csv.writer(csvfile, delimiter=';')
                for row in tsvreader:
                    new_row = []
                    for item in row:
                      # Split each item based on one or more spaces
                      items = re.split('\s+', item)
                      # Join the items using a comma separator
                      new_row.append(','.join(items))
                    csvwriter.writerow(new_row)


In [174]:
# Read in the csv files and create one 
# data frame per location and Tmean, Tmax, Tmin 
#locs= [ "Lagos" , "mavroneri" ]
names = [ "Tmax_2T_", "Tmean_2T_", "Tmin_2T_" ]
dfs={}
for item in locs: 
    count=0
    for name in names: 
        fname1= name+item+".csv"
        fname1b= name+item+"_COMMA.csv"
                #print(fname1)
        path1=os.path.join(path_era,fname1 )
        path1_csv = os.path.join(path_era,fname1b )
        column_names=[ 'empty1', 'empty2', 'empty3', 'empty4', 'empty5', 'name' , 'date', 'time', 'lon', 'lat', 'value', 'empty12'\
             , 'empty13', 'empty14', 'empty15', 'empty16', 'empty17']
        # Read the svaed comma delimeted csv with assigned column names
        #df2 = pd.read_csv(path1_csv, sep=',', names=column_names)
        c_name=name+item
        print("Data frame for: ", c_name)
        dfs[c_name]=pd.read_csv(path1_csv, sep=',', names=column_names)
        
        #Test commands to check if there are nan in columns (empty3 in this example) 
        #and count the number of nan per column.
        dfs[c_name]['empty3'].isnull().values.any()
        dfs[c_name]['empty3'].isnull().sum()
       # Drop the 'county_name' and 'state' columns
        dfs[c_name]=dfs[c_name].drop(['empty1', 'time', 'lon', 'lat', 'value', 'empty12', 'empty13', 'empty14', \
             'empty15', 'empty16', 'empty17'], axis='columns')
       # Drop the two rows that contains cdo output message from previous step 
        dfs[c_name] = dfs[c_name].tail(-2)
        # Rename columns
        dfs[c_name].rename(columns = {'empty2':'Name', 'empty3':'Date', 'empty4': 'time', \
              'empty5':'lon', 'name':'lat', 'date': c_name}, inplace = True)
        #print(dfs[c_name])
 




Data frame for:  Tmax_2T_Sitia
Data frame for:  Tmean_2T_Sitia
Data frame for:  Tmin_2T_Sitia


In [175]:
# Access the dataframes and concatenate for  the same location the Tmax, Tmin, Tmean 

#dfs['Tmax_2T_agioi10']
#locs =  [ "Lagos" , "mavroneri" ]
dfs_n={}
for name in locs: 
    #dfs_n[name] = pd.concat([dfs['Tmax_2T_agioi10'], dfs['Tmean_2T_agioi10']  ])
    new_name="Tmax_2T_"+name
    new_name2="Tmean_2T_"+name
    new_name3="Tmin_2T_"+name
    dfs[new_name].reset_index(drop=True, inplace=True)
    dfs[new_name2].reset_index(drop=True, inplace=True)
    dfs[new_name3].reset_index(drop=True, inplace=True)
    dfs_n[name] = pd.concat([dfs[new_name], dfs[new_name2], dfs[new_name3] ], axis=1) 
    # Drop duplicate columns 
    dfs_n[name] = dfs_n[name].T.drop_duplicates().T
#print(dfs_n)

 

#  Read in sarah satellite

In [176]:
# Convert to comma delimited csv 
## Convert all files from tsv to csv and save them locally with _COMMA.csv 
#merged_GHI_0_COMMA.csv
#locs= [ "Lagos" , "mavroneri" ]
count=0
for loc in locs: 
    for name in names: 
        count=count +1 
        count2=str(count)
        fname2= "merged_GHI_"+count2+".csv"
        fname2b= "merged_GHI_"+loc+"_COMMA.csv"
        #print(fname1)
        path2=os.path.join(path_sat,fname2 )
        path2_csv = os.path.join(path_sat,fname2b )
        #print(path1)
        #print(path1_csv)
        # Convert to comma delimited csv 
        with open(path2, 'r') as tsvfile, open(path2_csv, 'w', newline='') as csvfile:
                #print(tsvfile, name, loc)
                #print(csvfile, name, loc)
                tsvreader = csv.reader(tsvfile, delimiter='\t')
                csvwriter = csv.writer(csvfile, delimiter=';')
                for row in tsvreader:
                    new_row = []
                    for item in row:
                      # Split each item based on one or more spaces
                      items = re.split('\s+', item)
                      # Join the items using a comma separator
                      new_row.append(','.join(items))
                    csvwriter.writerow(new_row)

In [177]:
## Read in the csv nad pass it to dataframe per location

#locs= [ "Lagos" , "mavroneri" ]
dfs2={}
for item in locs: 
    count=0 
    fname2b= "merged_GHI_"+item+"_COMMA.csv"     
    path2_csv = os.path.join(path_sat,fname2b )
    import numpy as np
    # name the columns so that it can split each row accordingly 
    column_names=[ 'empty1', 'empty2', 'empty3', 'empty4', 'empty5', 'name' , 'date', 'time', 'lon', 'lat', 'value', 'empty12'\
             , 'empty13', 'empty14', 'empty15', 'empty16', 'empty17']
    # Read the svaed comma delimeted csv with assigned column names
    name=item 
    dfs2[name] = pd.read_csv(path2_csv, sep=',', names=column_names)
    # Test commands to check if there are nan in columns (empty3 in this example) 
    # and count the number of nan per column.
    dfs2[name]['empty3'].isnull().values.any()
    dfs2[name]['empty3'].isnull().sum()
    # Drop the 'county_name' and 'state' columns
    dfs2[name]=dfs2[name].drop(['empty1', 'empty2', 'time', 'lon', 'lat', 'value', 'empty12', 'empty13', 'empty14', \
             'empty15', 'empty16', 'empty17'], axis='columns')
    # Drop the two rows that contains cdo output message from previous step 
    dfs2[name] = dfs2[name].tail(-2)
    # Rename columns 
    name_solar="GHI"+item
    dfs2[name].rename(columns = {'empty3':'Date', 'empty4':'Time', 'empty5': 'lon', \
                      'name':'lat', 'date':name_solar}, inplace = True)

#dfs2

# Concatenate Satellite and Era5 

In [178]:
# Saellite dfs are named: dfs2 and ERA5: dfs_n
#locs= [ "Lagos" , "mavroneri" ]
df_SE={}
for name in locs: 
    dfs2[name].reset_index(drop=True, inplace=True)
    dfs_n[name].reset_index(drop=True, inplace=True)
    # Ensure the date columns are in datetime format]
    # Remove 'nearest' string Assuming df3 is your DataFrame
    dfs2[name] = dfs2[name][dfs2[name]['Date'] != 'Nearest']
    dfs_n[name] = dfs_n[name][dfs_n[name]['Date'] != 'Nearest']
    # Ensure the date column is in datetime format
    dfs_n[name]['Date'] = pd.to_datetime(dfs_n[name]['Date'])
    dfs2[name]['Date'] = pd.to_datetime(dfs2[name]['Date'])
    # Merge the two DataFrames only where the dates coincide
    df_SE[name] = pd.merge( dfs_n[name], dfs2[name], on='Date')
  
    # Drop uncessary columns 
    df_SE[name]=df_SE[name].drop(['Time', 'time', 'lon_y', 'lat_y', 'Name'], axis='columns')
    #print(df_SE)


#  Concatenate df_SE with on site data  : df_SEL

In [179]:
# # On site data:   
# path_site="/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SolarKapital/"
# path_out="/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SolarKapital/DATA_SK_1"


# # First merge the infividual site data 
# files = os.listdir(path_site)
# files_csv = [f for f in files if f[-3:] == 'csv']


In [180]:

import os

def find_directories_with_prefix(path, prefix):
    dirs = []
    for entry in os.listdir(path):
        full_entry_path = os.path.join(path, entry)
        #print(full_entry_path)
        if os.path.isdir(full_entry_path) and entry.startswith(prefix):
            dirs.append(full_entry_path)
            #print(full_entry_path)
    return dirs

# Usage
# The path to the parent directory you want to search in
#path_site='/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/SCRIPTS/ONSITE_DATA'
#prefix = "DATA_SK"  # The prefix you want to match

matching_directories = find_directories_with_prefix(path_site, prefix)
df_SK={}
df_SK2={}
df_names=[]
max_cols=[]
for directory in matching_directories:
    #print(f'Directory found: {directory}')
    names=os.path.basename(directory)
    #print(names)
    last_part = names.split('_', maxsplit=2)[2]
    df_names.append(last_part)
    files = os.listdir(directory)
    files_csv = [f for f in files if f[-3:] == 'csv']
    prefix=last_part
    path_out=os.path.join(path_site, prefix)
    df_SK[last_part] = pd.DataFrame()
    #print(last_part)
    for f in files_csv:
        f2=os.path.join(directory,f)
        #print(f2)
        data=pd.read_table(f2,  header=None, skiprows=1)
        df_SK[last_part] = pd.concat([df_SK[last_part], pd.DataFrame.from_records(data)])
    df_SK[last_part] = df_SK[last_part].reset_index(drop=True)
 

In [181]:
df_SK2={}
#names =  [ "Lagos" ]
for name in locs: 
    #print(df_SK[name], name)
    max_cols = max(df_SK[name].apply(lambda x: x.str.count(';').sum() + 1, axis=1))
    #print(max_cols)
    df_SK2[name] = pd.DataFrame(columns=range(max_cols))
    # Split each row into the corresponding number of columns
    for i, row in df_SK[name].iterrows():
            #print('Working on row', i)
            values = row.str.split(';', expand=True)
            #print(values)
            df_SK2[name].loc[i, :len(values.columns)-1] = values.values[0]
    # Rename columns 
    new_column_names = { 0: 'Date', 1: 'Yield(Kwh)'}   
    df_SK2[name] = df_SK2[name].rename(columns=new_column_names)        
    # Ensure the date column is in datetime format
    df_SK2[name]['Date'] = pd.to_datetime(df_SK2[name]['Date'])
    df_SK2[name] = df_SK2[name].sort_values(by='Date', ascending=False)
df_SK2

{'Sitia':            Date Yield(Kwh)
 916  2017-12-31    106.179
 915  2017-12-30    221.722
 914  2017-12-29    125.629
 913  2017-12-28    229.113
 912  2017-12-27     64.480
 ...         ...        ...
 610  2011-01-03    257.300
 824  2011-01-03    257.300
 0    2011-01-03    257.300
 1802 2011-01-01           
 1465 2011-01-01           
 
 [2354 rows x 2 columns]}

In [183]:
# Concartenate all
# Saellite dfs are named: dfs2 and ERA5: dfs_n
df_SEL={}
merged_df={}
data_frames={}
for name in locs:
    if name in df_SE and name in df_SK2:
        print("OK")
        df_SK2[name]['Date'] = pd.to_datetime(df_SK2[name]['Date'])
        # Merge the two DataFrames only where the dates coincide
        df_SEL[name] = pd.merge( df_SE[name], df_SK2[name], on='Date')     
        #df_SE[name]=df_SE[name].drop(['Time', 'time', 'lon_y', 'lat_y', 'Name'], axis='columns')
        
print(df_SEL)




OK
{'Sitia':            Date   lon_x    lat_x Tmax_2T_Sitia Tmean_2T_Sitia Tmin_2T_Sitia  \
0    2011-01-01  26.125  35.1316      11.75317       10.84762      10.16821   
1    2011-01-01  26.125  35.1316      11.75317       10.84762      10.16821   
2    2011-01-03  26.125  35.1316      12.91787       12.30602      11.26736   
3    2011-01-03  26.125  35.1316      12.91787       12.30602      11.26736   
4    2011-01-03  26.125  35.1316      12.91787       12.30602      11.26736   
...         ...     ...      ...           ...            ...           ...   
1647 2015-12-27  26.125  35.1316      15.06137       13.97433      13.05648   
1648 2015-12-28  26.125  35.1316      14.70553       13.86932       13.1332   
1649 2015-12-29  26.125  35.1316      14.12356       13.32867      12.67709   
1650 2015-12-30  26.125  35.1316      13.25308       11.70761      9.042475   
1651 2015-12-31  26.125  35.1316      8.441431       5.357202      3.832483   

     GHISitia Yield(Kwh)  
0         1

# Create labels for data and save the data into a csv 

In [184]:
# Get the upper and lower bounds  of yield data that are out in the 9 and 95 percintile.
# import the packages
import numpy as np
# chack the outliers for yield 
data=df_SEL['Sitia']['Yield(Kwh)']
data2=data.to_numpy()
# filter out empty strings 
data_rm = list(filter(None, data2))
arr = np.array(data_rm)
# Convert from string to float 
data3=arr.astype(float)

#print(np.where(data3))

q5, q95 = np.percentile(data3, [5, 95])
#print(q1, q3)
# iqr = q3 - q1
# lower_bound = q1 - 1.5*iqr
# upper_bound = q3 + 1.5*iqr
#print(data3)
lower_bound= q5
upper_bound=q95

# for index, element in enumerate(data3):
#     print(f"Index: {index}, Element: {element}")

outliers = np.where((data3 < lower_bound) | (data3 > upper_bound))


In [185]:
# Create labels for the data, 1 if the exceed the upper and lower bounds and 0 if they are within limits.
# Define a custom function that checks a condition and returns a value accordingly
def check_condition(row):
    #row['Yield(Kwh)'] = row['Yield(Kwh)'].astype(float)
    row['Yield(Kwh)'] = pd.to_numeric(row['Yield(Kwh)'])
    if row['Yield(Kwh)'] <= lower_bound:
        return 1
    elif row['Yield(Kwh)'] >= upper_bound:
        return 1
    else: 
        return 0
# Use the apply() function with the custom function to create a new column 'C'
df_SEL['Sitia']['label'] = df_SEL['Sitia'].apply(check_condition, axis=1)
 
#df_SEL['agioi10'][0:50]





In [187]:
#/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/DATA_FOR_TRAINING
df_SEL['Sitia'].to_csv('/Users/paraskevivourlioti/Documents/NEURALIO/KYKLOS/DATA_KYKLOS/DATA_FOR_TRAINING/SK_Sitia.csv', index=False)